In [16]:
import os, os.path
import random
from time import sleep
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization

In [17]:
path_e46 = 'C:\\Users\\mnaca\\Desktop\\BMW_3_e46'
path_f30 = 'C:\\Users\\mnaca\\Desktop\\BMW_3_f30'


In [21]:
class IMG_ACT():
    
    def __init__(self, *args):
        self.path = args
        self.__x = 460
        self.__y = 320
        self.__bmw_data = []
    
        
    def img_redact(self, img):
        img = cv2.imread(img)
        img = cv2.resize(img, (self.__x, self.__y))         #size based on data
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return img
    
    def img_add(self):
        for class_, path in enumerate(self.path):
            for image in os.listdir(path):
                self.__bmw_data.append((self.img_redact(os.path.join(path,image)), class_))
    
    def get_data(self):
        random.shuffle(self.__bmw_data)
        return self.__bmw_data
                
            

In [22]:
data = IMG_ACT(path_e46, path_f30)
data.img_add()

In [23]:
x_train, x_test, y_train, y_test = train_test_split([data.get_data()[i][0] for i in range(len(data.get_data()))],
                                                    [data.get_data()[i][1] for i in range(len(data.get_data()))],
                                                    test_size=0.2)


x_train_norm = [i / 255 for i in x_train]
x_test_norm = [i / 255 for i in x_test]

y_train_cat = keras.utils.to_categorical(y_train, 2)
y_test_cat = keras.utils.to_categorical(y_test, 2)

x_train_norm = np.expand_dims(x_train_norm, axis=3)
x_test_norm = np.expand_dims(x_test_norm, axis=3)


In [32]:
model = keras.Sequential([
    Conv2D(256, (32, 32), strides=(4, 4), padding='same', activation='relu', input_shape=(320, 460, 1)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(512, (4, 4), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(1024, (2, 2), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(512, (2, 2), padding='same', activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    # BatchNormalization(),
    Dense(128, activation='relu'),
    # Dropout(rate=0.45),
    Dense(2, activation='softmax')
])

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 80, 115, 256)      262400    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 40, 57, 256)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 40, 57, 512)       2097664   
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 20, 28, 512)      0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 20, 28, 1024)      2098176   
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 10, 14, 1024)    

In [42]:
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.1, nesterov=True),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
his = model.fit(x_train_norm, y_train_cat, batch_size=16, epochs=25, validation_split=0.2)

Epoch 1/25
8/8 [==============================] - 2s 254ms/step - loss: nan - accuracy: 0.4922 - val_loss: nan - val_accuracy: 0.4688
Epoch 2/25
8/8 [==============================] - 2s 230ms/step - loss: nan - accuracy: 0.4922 - val_loss: nan - val_accuracy: 0.4688
Epoch 3/25
8/8 [==============================] - 2s 229ms/step - loss: nan - accuracy: 0.4922 - val_loss: nan - val_accuracy: 0.4688
Epoch 4/25
8/8 [==============================] - 2s 230ms/step - loss: nan - accuracy: 0.4922 - val_loss: nan - val_accuracy: 0.4688
Epoch 5/25
8/8 [==============================] - 2s 229ms/step - loss: nan - accuracy: 0.4922 - val_loss: nan - val_accuracy: 0.4688
Epoch 6/25
8/8 [==============================] - 2s 231ms/step - loss: nan - accuracy: 0.4922 - val_loss: nan - val_accuracy: 0.4688
Epoch 7/25
8/8 [==============================] - 2s 229ms/step - loss: nan - accuracy: 0.4922 - val_loss: nan - val_accuracy: 0.4688
Epoch 8/25
8/8 [==============================] - 2s 229ms/ste

In [18]:
model.evaluate(x_test_norm, y_test_cat)

2/2 [==============================] - 2s 438ms/step - loss: 0.7092 - accuracy: 0.4000


[0.7092181444168091, 0.4000000059604645]